In [1]:
import pandas as pd
df = pd.read_csv('../small_10/statistical_interaction_data.csv')
df.head(20)

,user_id,date,app_id,is_recommended,cumulative_review,cumulative_recommended,cumulative_not_recommend,recommend_ratio
0,181,2014-01-02,245170,True,572,529,43,0.924825
1,181,2015-05-23,368950,True,241,224,17,0.929461
2,181,2015-08-11,221100,True,30136,23109,7027,0.766824
3,181,2015-12-17,376300,True,308,296,12,0.961039
4,181,2016-11-25,252490,True,36716,29771,6945,0.810845
5,181,2016-12-23,37000,True,215,192,23,0.893023
6,181,2017-11-23,391540,True,27700,25662,2038,0.926426
7,181,2018-11-22,253230,True,6996,6919,77,0.988994
8,181,2019-07-04,365590,True,43955,30294,13661,0.689205
9,181,2019-07-29,493900,True,3710,3512,198,0.946631


In [2]:
df['date'] = pd.to_datetime(df['date'])
latest_df = df.sort_values('date').groupby('app_id', as_index=False).last()
latest_df.head(5)


,app_id,user_id,date,is_recommended,cumulative_review,cumulative_recommended,cumulative_not_recommend,recommend_ratio
0,10,3468375,2022-12-31,True,41022,39311,1711,0.958291
1,20,10037716,2022-12-31,True,4277,3678,599,0.859949
2,30,10037716,2022-12-31,True,4429,3975,454,0.897494
3,40,10037716,2022-12-31,True,1608,1298,310,0.807214
4,50,910967,2022-12-31,True,9708,9273,435,0.955192


In [3]:
df.isnull().sum()

user_id                         0
date                            0
app_id                          0
is_recommended                  0
cumulative_review               0
cumulative_recommended          0
cumulative_not_recommend        0
recommend_ratio             76265
dtype: int64

In [5]:
latest_df.isnull().sum()
latest_df.fillna(0, inplace=True)
latest_df.isnull().sum()

app_id                      0
user_id                     0
date                        0
is_recommended              0
cumulative_review           0
cumulative_recommended      0
cumulative_not_recommend    0
recommend_ratio             0
dtype: int64

In [6]:
appid_dic = {}
for index, row in latest_df.iterrows():
    appid_dic[row['app_id']] = [row['cumulative_review'], row['cumulative_recommended'], row['cumulative_not_recommend'], row['recommend_ratio']]

    

In [7]:
userid_set = set(df['user_id'].unique())

In [27]:
# user_id = 181
# appid_user = df[df['user_id'] == user_id]['app_id'].unique()
# appid_user_set = set(appid_user)

In [11]:
import tqdm

grouped = df.groupby('user_id')['app_id'].unique()
userid_dic = {
    str(user): [int(appid) for appid in appids]  # 转换为原生 int，user 也最好转成 str
    for user, appids in tqdm.tqdm(grouped.items())
}


import json
with open('appid_user_dic.json', 'w') as f:
    json.dump(userid_dic, f)
    

274003it [00:01, 250165.24it/s]


In [8]:
appid_dic
#save the appid_dic to a json file
import json
with open('appid_dic.json', 'w') as f:
    json.dump(appid_dic, f)


In [1]:
import json

appid_read = {}
with open('appid_dic.json', 'r') as f:
    appid_read = json.load(f)

appid_read = {int(k): v for k, v in appid_read.items()}

userid_read = {}
with open('appid_user_dic.json', 'r') as f:
    userid_read = json.load(f)

userid_read = {int(k): v for k, v in userid_read.items()}

    
# 加载 gid2pos 映射
with open("../small_10/gid2pos_map.json", "r") as f:
    gid2pos = json.load(f)

# 将字符串形式的键转换为整数键（如果有必要）
gid2pos = {int(k): v for k, v in gid2pos.items()}

def convert_appid_to_pos(appid_list):
    """
    将一组 appid（原始游戏ID）转换为对应的 pos_id（模型内部ID）
    - appid_list: List[int] 原始的游戏 appid 列表
    - 返回 List[int] 映射后的 pos_id 列表（过滤非法ID）
    """
    return [gid2pos[appid] for appid in appid_list if appid in gid2pos]


In [2]:
import torch
import torch.nn as nn
import json
from model_din import DinAir  # 假设 model_din.py 中已经定义了 DinAir 模型

# 加载 game_cat_map
game_cat_map = {}
with open('../small_10/game_tags_mapping.json', 'r') as f:
    mapping_data = json.load(f)
    for key, value in mapping_data.items():
        game_cat_map[int(key)] = torch.zeros(442, dtype=torch.int).scatter(0, torch.tensor(value, dtype=torch.long), 1)
print("Game category map loaded.")

def inference(model, game_history, candidate_game_id, numerical_features):
    """
    推理函数：
    - model: 模型实例
    - game_history: List[int]，用户历史游戏 ID
    - candidate_game_id: int，候选游戏 ID
    - numerical_features: List[float]，数值特征（例如：[num_feature1, num_feature2, num_feature3, num_feature4]）
    
    返回值：预测的数值概率（经过 sigmoid 函数映射）
    """
    model.eval()
    # max_id = model.item_emb.num_embeddings
    # if candidate_game_id >= max_id or any(gid >= max_id for gid in game_history):
    #     print(f"⚠️ ID 超过 item_num 限制：最大允许 {max_id-1}，candidate_game_id 是{candidate_game_id}")
    #     return 0.0  # 或者 return None / raise Error
    # print(game_history, type(game_history))
    # print(candidate_game_id, type(candidate_game_id))
    # print(numerical_features, type(numerical_features))
    # return 0.0
    with torch.no_grad():
        # 候选游戏ID转换为tensor，并从game_cat_map中获取对应的游戏类别向量
        candidate_game_id_tensor = torch.tensor([candidate_game_id], dtype=torch.long)
        candidate_game_cats = game_cat_map.get(candidate_game_id, torch.zeros(442, dtype=torch.int)).unsqueeze(0).float()
        
        # 数值特征 tensor，shape: [1, 特征数]
        item_num_features = torch.tensor(numerical_features, dtype=torch.float).unsqueeze(0)
        
        # 将用户历史游戏ID转换为 tensor，shape: [1, T]
        hist_item_ids = torch.tensor(game_history, dtype=torch.long).unsqueeze(0)
        
        # 为每个历史游戏从game_cat_map中获取类别向量，stack后 shape: [1, T, 442]
        history_cats = torch.stack([
            game_cat_map.get(game_id, torch.zeros(442, dtype=torch.int)) for game_id in game_history
        ]).unsqueeze(0).float()
        
        # 序列掩码，由于这里未做padding，因此全为False，shape: [1, T]
        seq_mask = torch.zeros(hist_item_ids.shape, dtype=torch.bool)
        
        # 模型 forward 的输入顺序需与训练时保持一致：
        # candidate_game_id_tensor, candidate_game_cats, item_num_features, hist_item_ids, history_cats, seq_mask
        output = model(
            candidate_game_id_tensor,
            candidate_game_cats,
            item_num_features,
            hist_item_ids,
            history_cats,
            seq_mask
        )
        # 输出经过 sigmoid 映射为概率
        prob = torch.sigmoid(output.squeeze()).item()
    return prob

# def get_recommendations(userid):
#     model = DinAir(
#         item_num=18369,
#         cat_num=442,
#         num_feature_size=4,
#         hidden_size=64,
#         num_heads=2
#     )
#     params_path = "../models/din_air.pt"
#     # model.load_state_dict(torch.load(params_path))
#     model.load_state_dict(torch.load(params_path, map_location=torch.device('cpu')))

    
#     # 注意：此处未加载预训练权重，实际使用时可加载保存的模型 checkpoint

#     rec_tuple = []
#     game_history = convert_appid_to_pos(userid_read[userid])
#     for appid in appid_read.keys():
#         # print(type(appid))
#         if appid not in userid_read[userid]:
#             candidate_game_id = gid2pos[appid]
#             numerical_features = appid_read[appid]
#             probability = inference(model, game_history, candidate_game_id, numerical_features)
#             rec_tuple.append((appid, probability))
#         # break
#     rec_tuple = sorted(rec_tuple, key=lambda x: x[1], reverse=True)
#     # rec_tuple = rec_tuple[:10]
#     print(rec_tuple[:10])

def get_recommendations(userid, topk=10):
    model = DinAir(
        item_num=18369,
        cat_num=442,
        num_feature_size=4,
        hidden_size=64,
        num_heads=2
    )
    params_path = "../models/din_air.pt"
    model.load_state_dict(torch.load(params_path, map_location=torch.device('cpu')))

    rec_tuple = []
    user_history_appids = set(userid_read[userid])
    game_history = convert_appid_to_pos(userid_read[userid])

    for appid in appid_read:
        if appid not in user_history_appids and appid in gid2pos:
            candidate_game_id = gid2pos[appid]
            numerical_features = appid_read[appid]
            probability = inference(model, game_history, candidate_game_id, numerical_features)
            rec_tuple.append((appid, probability))

    rec_tuple = sorted(rec_tuple, key=lambda x: x[1], reverse=True)
    return rec_tuple[:topk]


        

Game category map loaded.


In [3]:
get_recommendations(181)

[(550340, 0.9932155609130859),
 (716710, 0.9900977611541748),
 (329490, 0.9885492920875549),
 (452510, 0.9871342778205872),
 (370460, 0.9851206541061401),
 (411830, 0.9837402701377869),
 (318530, 0.9833920001983643),
 (657810, 0.9806817173957825),
 (388750, 0.9794080257415771),
 (411370, 0.9767463207244873)]

In [ ]:
[(550340, 0.9932155609130859), (452510, 0.9871342778205872), (318530, 0.9833920001983643), (294810, 0.9761815071105957), (263300, 0.9695678353309631), (302790, 0.965370237827301), (222420, 0.9645651578903198), (314180, 0.9498965740203857), (313980, 0.9474478960037231), (71260, 0.9432657957077026)]


In [6]:
import sqlite3

user_id = 76561199072485506

with sqlite3.connect('../user_info.db') as conn:
    cursor = conn.cursor()
    
    # 删除历史记录
    cursor.execute('DELETE FROM histories WHERE user_id = ?', (user_id,))
    
    # 删除推荐记录
    cursor.execute('DELETE FROM recommendations WHERE user_id = ?', (user_id,))
    
    # 最后删除用户记录
    cursor.execute('DELETE FROM users WHERE user_id = ?', (user_id,))
    
    conn.commit()
